In [1]:
import numpy as np

In [2]:
with open("likelihood.txt", "r") as f:
    bird_likelihood, aero_likelihood = f.readlines()

bird_likelihood = np.array([float(datum) for datum in bird_likelihood.split()])
aero_likelihood = np.array([float(datum) for datum in aero_likelihood.split()])
len(bird_likelihood), len(aero_likelihood)

(400, 400)

In [3]:
with open("dataset.txt", "r") as f:
    dataset = f.read()
    
dataset = np.array([[float(datum) for datum in row.split()] for row in dataset.strip().split("\n")])
bird_velocities = dataset[:10]
aero_velocities = dataset[10:]

len(bird_velocities[0]), len(aero_velocities[-1])

(600, 600)

In [4]:
with open("testing.txt", "r") as f:
    testing = f.read()

testing = np.array([[float(datum) for datum in row.split()] for row in testing.strip().split("\n")])
bird_test = testing[:5]
aero_test = testing[5:]

len(bird_test[0]), len(aero_test[0])

(600, 600)

In [5]:
# P(speed | b): probability of observing a bird's speed
# P(speed | a): probability of observing a airplane's speed

# P(a_t+1 | a_t) = .9; P(b_t+1 | b_t) = .9
# P(a_t+1 | b_t) = .1; P(b_t+1 | a_t) = .1

# P(a) = .5
# P(b) = .5

In [6]:
# for a single obs
min_speed = np.min(bird_velocities[0])
max_speed = np.max(bird_velocities[0])
normalized_speeds = (bird_velocities[0] - min_speed)/(max_speed - min_speed)

indices = (normalized_speeds * (len(bird_likelihood)-1)).astype(int)
first_row_likelihoods = bird_likelihood[indices]

In [7]:
def speed2likelihood(speeds, bird=True):
    indices = (speeds * 2).astype(int)
    max_likelihood_index = len(bird_likelihood) - 1
    indices = np.clip(indices, 0, max_likelihood_index)

    if bird:
        return bird_likelihood[indices]
    else:
        return aero_likelihood[indices]

In [8]:
test_bird_row = bird_velocities[0]
test_aero_row = aero_velocities[0]
test_row_bird_likelihoods = speed2likelihood(test_bird_row)
test_row_aero_likelihoods = speed2likelihood(test_aero_row, bird=False)

In [9]:
i = 0

# observed likelihoods
obs_likelihood_bird = test_row_bird_likelihoods[i]
obs_likelihood_aero = test_row_aero_likelihoods[i]

# priors
prob_bird = .5
prob_aero = .5

In [10]:
# posterior update
# p(b /\ speed) = p(speed | b) * p(b)
# p(a /\ speed) = p(speed | a) * p(a)

bird_post_update = obs_likelihood_bird * prob_bird
aero_post_update = obs_likelihood_aero * prob_aero

# normalization
bird_normed = bird_post_update/(bird_post_update + aero_post_update)
aero_normed = aero_post_update/(bird_post_update + aero_post_update)

In [11]:
bird_normed, aero_normed

(np.float64(0.9030574638043971), np.float64(0.09694253619560289))

In [12]:
# iteration 2
i = 1

# observed likelihoods
obs_likelihood_bird = test_row_bird_likelihoods[i]
obs_likelihood_aero = test_row_aero_likelihoods[i]

# transition probabilities
prob_bird = bird_normed * .9 + aero_normed * .1
prob_aero = aero_normed * .9 + bird_normed * .1

In [13]:
bird_post_update = obs_likelihood_bird * prob_bird
aero_post_update = obs_likelihood_aero * prob_aero

bird_normed = bird_post_update/(bird_post_update + aero_post_update)
aero_normed = aero_post_update/(bird_post_update + aero_post_update)

In [14]:
bird_normed, aero_normed

(np.float64(0.9564670147962513), np.float64(0.043532985203748695))

In [23]:
prob_bird = .5
prob_aero = .5

for j in range(len(bird_velocities)):
    print(j)
    bird_row = bird_velocities[j]
    aero_row = aero_velocities[j]

    row_bird_likelihoods = speed2likelihood(bird_row)
    row_aero_likelihoods = speed2likelihood(aero_row, bird=False)

    for i in range(len(row_bird_likelihoods)):
        obs_likelihood_bird = row_bird_likelihoods[i]
        obs_likelihood_aero = row_aero_likelihoods[i]

        if not obs_likelihood_aero:
            continue
        if not obs_likelihood_bird:
            continue

        if i != 0:
            prob_bird = bird_normed * .9 + aero_normed * .1
            prob_aero = aero_normed * .9 + bird_normed * .1

        bird_post_update = obs_likelihood_bird * prob_bird
        aero_post_update = obs_likelihood_aero * prob_aero

        bird_normed = bird_post_update/(bird_post_update + aero_post_update)
        aero_normed = aero_post_update/(bird_post_update + aero_post_update)

0
1
2
3
4
5
6
7
8
9


/tmp/ipykernel_10321/154105515.py:2: RuntimeWarning: invalid value encountered in cast
  indices = (speeds * 2).astype(int)


In [20]:
bird_normed, aero_normed

(np.float64(0.8088557341316833), np.float64(0.1911442658683167))